In [1]:
%matplotlib widget

In [2]:
from ase.visualize import view

In [3]:
from genSTEM import ImageModel
import cupy as cp
import numpy as np
import matplotlib.pyplot as plt
from ase.io import read
from ase.build import make_supercell
atoms = read(r"C:\Users\thomasaar\Downloads\imeall-master\imeall\examples\boundary_interstitials\0013687130_v6bxv2_tv0.1bxv0.0_d1.8z_traj.xyz")
atoms = make_supercell(atoms, np.diag((1,2,1)))
mask = atoms.positions[:,2] > 35
del atoms[mask]
atoms.rotate([1,0,0], (0,0,1))

In [4]:
atoms[1716].number = 60
atoms[1766].number = 60

In [6]:
%%time
jitter = 0

drift_strength = 5e-5
drift_vector = [-1,0]
centre_drift = True

m0 = ImageModel(atoms, 
                scan_rotation=0, centre_drift=centre_drift)
img0 = m0.generate_cupy()

m1 = ImageModel(atoms, 
                drift_strength=drift_strength, 
                drift_vector=drift_vector,
                jitter_strength=jitter, 
                scan_rotation=0, centre_drift=centre_drift)
img1 = m1.generate_cupy()

m2 = ImageModel(atoms, 
                jitter_strength=jitter, 
                drift_vector=drift_vector,
                drift_strength=drift_strength, 
                scan_rotation=90, centre_drift=centre_drift)
img2 = m2.generate_cupy()




Wall time: 2.46 s


In [7]:
%%time
img2 = cp.rot90(img2, 3)
side = np.minimum(*img1.shape)

img1 = img1[:side, :side]
img2 = img2[:side, :side]

Wall time: 0 ns


In [8]:
I0, I1, I2 = img0.get(), img1.get(), img2.get()

In [39]:
from skimage.transform import AffineTransform
from cupyx.scipy.ndimage import affine_transform
n = 1000
N = np.linspace(-n, n, 11)
w = img0.shape[0]
scales = 1 + N / w
shears = np.arctan(N/w)

shifts = np.array(img0.shape) / 2.0
shift = AffineTransform(translation=-shifts)
shift_inv = AffineTransform(translation=shifts)


def get_matrix(transform):
    return np.linalg.inv(transform.params)

transform1 = np.array(
    [get_matrix(shift + AffineTransform(shear=shear) + shift_inv) 
        for shear, scale in zip(shears, scales)])
transform1 = cp.array(transform1)

transform2 = np.array(
    [get_matrix(shift + AffineTransform(scale=scale) + shift_inv) 
        for shear, scale in zip(shears, scales)])
transform2 = cp.array(transform2)


In [40]:
def plot(imgs):
    imgs2 = []
    for img in imgs:
        img -= img.min()
        img /= img.max()
        img *= 255
        img = fromarray(img.astype("uint8"))
        imgs2.append(img)
    display(TwoByTwoLayout(imgs2))

In [41]:
from ipywidgets import TwoByTwoLayout, Output
from PIL.Image import fromarray

def plot(img):
    img -= img.min()
    img /= img.max()
    img *= 255
    img = fromarray(img.astype("uint8"))
    display(img)

In [42]:
from ipywidgets import GridspecLayout

def subplots(imgs, ncols=2):
    nrows = int(np.ceil(len(imgs) / ncols))
    grid = GridspecLayout(nrows, ncols)
    for i in range(nrows):
        for j in range(ncols):
            index = i*nrows + j
            if index < len(imgs):
                o = Output()
                with o:
                    plot(imgs[index])
                grid[i, j] = o
                
subplots([I0, I1])

In [44]:
fig = plt.figure(figsize = (5, 20))
grid = plt.GridSpec(ncols=2, nrows = len(shears), figure=fig)
for row in range(len(shears)):
    ax1, ax2 = fig.add_subplot(grid[row, 0]), fig.add_subplot(grid[row, 1])
    ax1.imshow(affine_transform(img1, transform1[row]).get())
    ax2.imshow(affine_transform(img2, transform2[row]).get())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [112]:
fig, (ax1, ax2, ax3) = plt.subplots(ncols=3)
ax1.imshow(I0)
ax2.imshow(I1)
ax3.imshow(I2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [92]:
nrows = 5
ncols = 5
img = I1

shift_x, shift_y = np.array(img.shape) / 2.0
tf_shift = AffineTransform(translation=[-shift_y, -shift_x])
tf_shift_inv = AffineTransform(translation=[shift_y, shift_x])


fig, AX = plt.subplots(ncols=ncols, nrows=nrows, )
shears = np.linspace(-np.pi, np.pi, np.prod((ncols, nrows)))

for shear, ax in zip(shears, AX.flatten()):
    ax.imshow(warp(img, tf_shift + AffineTransform(shear=shear) + tf_shift_inv))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [55]:
shears

array([-3.14159265, -2.87979327, -2.61799388, -2.35619449, -2.0943951 ,
       -1.83259571, -1.57079633, -1.30899694, -1.04719755, -0.78539816,
       -0.52359878, -0.26179939,  0.        ,  0.26179939,  0.52359878,
        0.78539816,  1.04719755,  1.30899694,  1.57079633,  1.83259571,
        2.0943951 ,  2.35619449,  2.61799388,  2.87979327,  3.14159265])

In [20]:
from skimage.transform import AffineTransform, warp

shear_limits = 0.8
shear_step = 0.01
scale_limits = 0.8
scale_step = 0.01

import ipywidgets.widgets as w
SCALE = w.FloatSlider(value=1, min=-scale_limits + 1, max=scale_limits + 1, step=shear_step, description='Scale')
SHEAR = w.FloatSlider(value=0, min=-shear_limits, max=shear_limits, step=scale_step, description='Shear')
img = I2

o = w.Output()
def warpit(img):
    angle = 0
    rot_matrix = AffineTransform(rotation = np.deg2rad(-angle))
    shear_matrix = AffineTransform(shear = SHEAR.value,)
    scale_matrix = AffineTransform(scale = (1, SCALE.value))

    shift_x, shift_y = np.array(img.shape) / 2.0
    tf_shift = AffineTransform(translation=[-shift_y, -shift_x])
    tf_shift_inv = AffineTransform(translation=[shift_y, shift_x])
    warped = warp(img,(tf_shift + shear_matrix + scale_matrix + rot_matrix + tf_shift_inv).inverse, order=0,)
    return warped

fig, (ax1, ax2) = plt.subplots(ncols=2)

ax1.imshow(I1)
im = ax2.imshow(warpit(img))

def on_value_change(change):
    with o:
        new_img = warpit(img)
        im.set_data(new_img)
        im.autoscale()

SCALE.observe(on_value_change, names='value')
SHEAR.observe(on_value_change, names='value')
display(SCALE)
display(SHEAR)
display(o)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

FloatSlider(value=1.0, description='Scale', max=1.8, min=0.19999999999999996, step=0.01)

FloatSlider(value=0.0, description='Shear', max=0.8, min=-0.8, step=0.01)

Output()

In [23]:
from PIL.Image import fromarray

In [46]:
def plot(img):
    img -= img.min()
    img /= img.max()
    img *= 255
    img = fromarray(img.astype("uint8"))
    display(img)

In [48]:
o1 = Output()
o2 = Output()
with o1:
    plot(I1)
with o2:
    plot(I2)

In [53]:
from ipywidgets import Output, VBox, HBox, TwoByTwoLayout
TwoByTwoLayout(top_left=VBox((HBox((o1, o1)), HBox((o1, o1)))), top_right=o2)

TwoByTwoLayout(children=(VBox(children=(HBox(children=(Output(layout=Layout(grid_area='top-left'), outputs=({'…

In [21]:
fig, (ax1, ax2) = plt.subplots(ncols=2)
ax1.imshow(img1.get())
ax2.imshow(img2.get())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [29]:
hamm = np.hamming(img1.shape[0])
window = hamm[:, None] * hamm

def hybrid_correlation(img1, img2, p=0.9, filter=True):
    if filter:
        hamm = cp.hamming(img1.shape[0])
        window = hamm[:, None] * hamm
        img1 = img1 * window
        img2 = img2 * window

    fftimg1 = cp.fft.fft2(img1)
    fftimg2 = cp.fft.fft2(img2)
    m = fftimg1 * cp.conj(fftimg2)
    corr =  cp.real(cp.fft.ifft2(cp.abs(m)**p * cp.exp(1j * cp.angle(m))))
    corr = cp.fft.fftshift(corr)
    translation = cp.array(cp.unravel_index(corr.argmax(), corr.shape))
    center = cp.array(corr.shape) // 2
    return tuple(x for x in translation - center)

def translate(img, shift):
    return cp.roll(img, shift, axis=(0,1))

In [84]:
img2b = translate(img2, hybrid_correlation(img1, img2))
plt.figure()
plt.imshow((img1 + img2b).get())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [78]:
I2 = img1.get()
I2 = img2.get()

In [24]:
from scipy.ndimage import fourier_shift

In [56]:
I1, I2 = img1.get(), img2.get()
I3 = np.fft.ifft2(fourier_shift(np.fft.fft2(I2), S)).real
I4 = np.roll(I2, S[0], axis=0)
I4 = np.roll(I4, S[1], axis=1)
I4 = np.roll(I2, S, axis=(0,1))
I5 = scipy.ndimage.shift(I2, S)

plt.figure()
plt.imshow(I1 + I4)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [55]:
I4 = np.roll(I2, S, axis=(0,1))


In [29]:
plt.figure()
plt.imshow(I2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
corr = scipy.signal.correlate2d(img1.get(), img2.get())

In [237]:
plt.figure()
plt.imshow(window)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
window = 

In [13]:
data = np.random.random((300, 300))
%timeit np.real(np.fft.ifft(data))
%timeit np.fft.irfft(data)

957 µs ± 22.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
2.92 ms ± 84.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [18]:
from py4DSTEM.process.utils import get_maxima_2D

In [ ]:
get_maxima_2D()

In [19]:
# a = np.real(np.fft.ifft(data))
# b = np.fft.irfft(data)
# np.testing.assert_array_almost_equal(a, b)

In [15]:
a.shape

(300, 300)

In [16]:
b.shape

(300, 598)

In [5]:
import matplotlib.pyplot as plt
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(10,10))
ax1.imshow(cp.asnumpy(img1))
ax2.imshow(cp.asnumpy(cp.rot90(img2, 3)))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
m.generate_cupy(jitt)